In [71]:
import numpy as np
import itertools
import pandas as pd

import pickle

#!pip install sentence_transformers
#!pip install konlpy
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [58]:
df = pd.read_csv('../Common/data/data/dataset/CAI_data_005.csv')

In [59]:
df.head()

,본부,VOC 유형,sentence
0,서울남부,부팅중 오류메시지 발생,차동일 요청사항리셋 접수 수신 불수산 카드 오류리 셋
1,전북,장비리셋 후 재사용 요청,리셋 접수 채널 수신 불량
2,인천,수신 불량,특정 채널 수신 불법 나옴 재부팅하면 사용 가능하다가 다시
3,경기남부,장비리셋 후 재사용 요청,리셋 접수 채널 수신 불량 인터넷 연결 오류
4,충남,수신 불량,리셋 연락 우회선


In [60]:
print(len(df))
df.head()

20000


,본부,VOC 유형,sentence
0,서울남부,부팅중 오류메시지 발생,차동일 요청사항리셋 접수 수신 불수산 카드 오류리 셋
1,전북,장비리셋 후 재사용 요청,리셋 접수 채널 수신 불량
2,인천,수신 불량,특정 채널 수신 불법 나옴 재부팅하면 사용 가능하다가 다시
3,경기남부,장비리셋 후 재사용 요청,리셋 접수 채널 수신 불량 인터넷 연결 오류
4,충남,수신 불량,리셋 연락 우회선


In [61]:
doc = df['sentence']

In [62]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [63]:
tokenized_docs = []
no_token = []
tokenized_nounses = []
no_noun = []


for i in range(len(doc)) :
  okt = Okt()
  try :
    tokenized_doc = okt.morphs(doc[i])
  except :
    print('비어있는 컬럼')
  if len(tokenized_doc) >= 2 : 
    tokenized_docs.append(tokenized_doc)

# tokenized_nouns = ' '.join([word[0] for word in tokenized_docs if word[1] == 'Noun'])
# if len(tokenized_doc) >= 2 :
#   tokenized_nounses.append(tokenized_nouns)

비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼
비어있는 컬럼


In [64]:
tokenized_nounses = []
for word_list in tokenized_docs:
  preprocess_list = []
  for word in word_list:
    preprocess_list.append(word)
  tokenized_nounses.append([' '.join(preprocess_list)])

In [65]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [66]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
keywords_list = []
for i in range(len(tokenized_nounses)) :
  try:
    n_gram_range = (0, 1)
    count = CountVectorizer(ngram_range=n_gram_range).fit(tokenized_nounses[i])
    candidates = count.get_feature_names()

    doc_embedding = model.encode([tokenized_nounses])
    candidate_embeddings = model.encode(candidates)

    top_n = 1
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    #keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=1, nr_candidates=10)
    #keywords = mmr(doc_embedding, candidate_embeddings, candidates, top_n=1, diversity=0.7)
    print('원본 :', doc[i]) 
    print('문장의 키워드 :' , keywords,)
    keywords_list.append(keywords)
        

    print('trigram 개수 :',len(candidates))
    print('후보군 단어 출력 :',candidates[:5])
    print('------------------------------------------------------------------------------------------------------') 
  except ValueError as e:
    print('불용어만 있습니다', e)
    print('------------------------------------------------------------------------------------------------------') 
  except TypeError as c:
    print('Nonetype이 존재합니다', c)
    print('------------------------------------------------------------------------------------------------------') 

with open('../Keywords/Keywords_list.pickle', 'wb') as f:
    pickle.dump(keywords_list, f, pickle.HIGHEST_PROTOCOL)

In [68]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['버튼', '안내', '리셋', '작동', '현재']